In [ ]:
from sklearn.metrics import mean_absolute_error
from heartpy.preprocessing import flip_signal
from neurokit2.ppg import ppg_findpeaks
from tqdm import tqdm

sbp_err = []
dbp_err = []
sbp = []
sbp_predictions = []
dbp = []
dbp_predictions = []
for t, p in tqdm(zip(true, pred), total=true.shape[0]):
    try:
        t_peaks = ppg_findpeaks(t, sampling_rate=125)['PPG_Peaks']
        p_peaks = ppg_findpeaks(p, sampling_rate=125)['PPG_Peaks']
        t_valleys = ppg_findpeaks(flip_signal(t), sampling_rate=125)['PPG_Peaks']
        p_valleys = ppg_findpeaks(flip_signal(p), sampling_rate=125)['PPG_Peaks']

        sbp_true = np.mean(t[t_peaks])
        dbp_true = np.mean(t[t_valleys])
        sbp_pred = np.mean(p[p_peaks])
        dbp_pred = np.mean(p[p_valleys])

        sbp.append(sbp_true)
        sbp_predictions.append(sbp_pred)

        dbp.append(dbp_true)
        dbp_predictions.append(dbp_pred)

        sbp_err.append(mean_absolute_error(sbp_true.reshape(-1, 1), sbp_pred.reshape(-1, 1)))
        dbp_err.append(mean_absolute_error(dbp_true.reshape(-1, 1), dbp_pred.reshape(-1, 1)))
    except:
        continue
sbp_err = np.array(sbp_err)
dbp_err = np.array(dbp_err)

In [ ]:
fig = pd.DataFrame(dict(true=sbp, pred=sbp_predictions)).plot.scatter(x='true', y='pred')
fig.update_layout(
    xaxis=dict(
        range=[0, 230],
    ),
    yaxis=dict(
        range=[0, 230],
    ),
)

In [ ]:
import numpy as np
import plotly.graph_objects as go

def bland_altman_plot(data1, data2, *args, **kwargs):
    data1     = np.asarray(data1)
    data2     = np.asarray(data2)
    mean      = np.mean([data1, data2], axis=0)
    diff      = data1 - data2                   # Difference between data1 and data2
    md        = np.mean(diff)                   # Mean of the difference
    sd        = np.std(diff, axis=0)            # Standard deviation of the difference

    fig = go.Figure()
    fig.add_scatter(x=mean, y=diff, mode='markers')
    fig.add_hline(y=md)
    for i in [5, 10, 15]:
        fig.add_hline(y=md + i, line_dash='dot')
        fig.add_hline(y=md - i, line_dash='dot')
    fig.update_layout(
        yaxis=dict(
            range=[-55, 55],
        )
    )
    return fig

fig = bland_altman_plot(dbp, dbp_predictions)
fig

In [ ]:
fig = pd.Series(sbp_err).plot.hist(nbins=100)
print(f'{len(sbp_err[sbp_err < 15]) / len(sbp_err) * 100:.{3}}% < 15mmHg')
print(f'{len(sbp_err[sbp_err < 10]) / len(sbp_err) * 100:.{3}}% < 10mmHg')
print(f'{len(sbp_err[sbp_err < 5]) / len(sbp_err) * 100:.{3}}% < 5mmHg')
fig

In [ ]:
fig = pd.Series(dbp_err).plot.hist(nbins=100)
print(f'{len(dbp_err[dbp_err < 15]) / len(dbp_err) * 100:.{3}}% < 15mmHg')
print(f'{len(dbp_err[dbp_err < 10]) / len(dbp_err) * 100:.{3}}% < 10mmHg')
print(f'{len(dbp_err[dbp_err < 5]) / len(dbp_err) * 100:.{3}}% < 5mmHg')
fig